In [16]:
#pip install spacy
#!python -m spacy download en_core_web_lg

#!python -m spacy download en

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import warnings
warnings.filterwarnings('ignore')
os.chdir("C:/Users/Abhishek/Desktop/Case Studies/AI ML/Quora Question Pair")

import spacy
from tqdm import tqdm

<h2>Featurizing text data with tfidf weighted word-vectors </h2>

In [18]:
data=pd.read_csv('train.csv')
data.head(3)

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0


In [19]:
#data.dtypes
# Need to convert Q1 and Q2 to string

#data["question1"]=data["question1"].apply(lambda x : str(x))
#data["question2"]=data["question2"].apply(lambda x : str(x))

In [20]:
data=data.dropna()
data.isna().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [21]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

# Merge All texts i.e Q1 and Q2
Questions= list(data["question1"]) + list(data["question2"]) 

#Initialise TFIDF, and Fit and Transform it on "Questions"
tfidf = TfidfVectorizer(lowercase =False)
tfidf.fit_transform(Questions)


# Creat a Dict with - key:word and value:tf-idf score i.e tfidf.get_feature_names is Key and tfidf.idf_ is Value
word2tfidf = dict(zip(tfidf.get_feature_names(), tfidf.idf_))

### INstead of W2V we will be using GLOVE, it is similar to W2V i.e given a word it returns a vector.
### But it is stronger as it is trained on Wikipedia data.

- After we find TF-IDF scores, we convert each question to a weighted average of word2vec vectors by these scores.
- Here we use a pre-trained GLOVE model which comes free with "Spacy".  https://spacy.io/usage/vectors-similarity
- It is trained on Wikipedia and therefore, it is stronger in terms of word semantics. 

In [22]:
# en_vectors_web_lg, which includes over 1 million unique vectors.
#import spacy

#nlp = spacy.load('en')

nlp = spacy.load('en_core_web_sm') #To use GLOVE inplace of W2V

vecs1 = []
# https://github.com/noamraph/tqdm
# tqdm is used to print the progress bar
for qu1 in tqdm(list(data['question1'])):
    doc1 = nlp(qu1) 
    # 384 is the number of dimensions of vectors 
    mean_vec1 = np.zeros([len(doc1), len(doc1[0].vector)]) #Create vector with 0's of appropriate dimensions
    for word1 in doc1:
        # word2vec
        vec1 = word1.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word1)]  ##Fetching tfidf Score from word2tfidf
        except:
            idf = 0
        # compute final vec
        mean_vec1 += vec1 * idf
    mean_vec1 = mean_vec1.mean(axis=0)
    vecs1.append(mean_vec1)
data['q1_feats_m'] = list(vecs1)


100%|█████████████████████████████████████████████████████████████████████████| 404287/404287 [59:20<00:00, 113.55it/s]


In [ ]:
vecs2 = []
mean_vec2=0
for qu2 in tqdm(list(data['question2'])):
    doc2 = nlp(qu2) 
    mean_vec1 = np.zeros([len(doc1), len(doc2[0].vector)])
    for word2 in doc2:
        # word2vec
        vec2 = word2.vector
        # fetch df score
        try:
            idf = word2tfidf[str(word2)]
        except:
            #print word
            idf = 0
        # compute final vec
        mean_vec2 += vec2 * idf
    mean_vec2 = mean_vec2.mean(axis=0)
    vecs2.append(mean_vec2)
data['q2_feats_m'] = list(vecs2)

 15%|███████████▎                                                             | 62367/404287 [12:22<2:18:58, 41.01it/s]

In [ ]:
#prepro_features_train.csv (Simple Preprocessing Feartures)
#nlp_features_train.csv (NLP Features)
#Read Both of these Files

if os.path.isfile('nlp_features_train.csv'):
    dfnlp = pd.read_csv("nlp_features_train.csv",encoding='latin-1')
else:
    print("download nlp_features_train.csv from drive or run previous notebook")

if os.path.isfile('df_fe_without_preprocessing_train.csv'):
    dfppro = pd.read_csv("df_fe_without_preprocessing_train.csv",encoding='latin-1')
else:
    print("download df_fe_without_preprocessing_train.csv from drive or run previous notebook")

In [ ]:
df1 = dfnlp.drop(['qid1','qid2','question1','question2'],axis=1)                  # From NLP_features
df2 = dfppro.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)  # From pre_processing features
df3 = data.drop(['qid1','qid2','question1','question2','is_duplicate'],axis=1)      # From Current DF
df3_q1 = pd.DataFrame(df3.q1_feats_m.values.tolist(), index= df3.index)           # df3["q1_feats_m"]
df3_q2 = pd.DataFrame(df3.q2_feats_m.values.tolist(), index= df3.index)           # df3["q2_feats_m"]

In [ ]:
# dataframe of nlp features
df1.head()

In [ ]:
# data before preprocessing 
df2.head()

In [ ]:
# Questions 1 tfidf weighted word2vec
df3_q1.head()

In [ ]:
# Questions 2 tfidf weighted word2vec
df3_q2.head()

In [ ]:
print("Number of features in nlp dataframe :", df1.shape[1])
print("Number of features in preprocessed dataframe :", df2.shape[1])
print("Number of features in question1 w2v  dataframe :", df3_q1.shape[1])
print("Number of features in question2 w2v  dataframe :", df3_q2.shape[1])
print("Number of features in final dataframe  :", df1.shape[1]+df2.shape[1]+df3_q1.shape[1]+df3_q2.shape[1])

In [ ]:
# storing the final features to csv file
if not os.path.isfile('final_features.csv'):
    df3_q1['id']=df1['id']
    df3_q2['id']=df1['id']
    df1  = df1.merge(df2, on='id',how='left')        # a to b
    df2  = df3_q1.merge(df3_q2, on='id',how='left')  # b to c
    result  = df1.merge(df2, on='id',how='left')     #a to b (where b is now b+c). Therefore we have abc
    result.to_csv('final_features_try.csv')

In [ ]:
data['q1_feats_m'].head(5)  
#For Every Sentence : It will Return a Score for Each word i.e(returns a Vector for each word)
# The Semantic Relationship of the words is preserved in this vector

# Here we find Vector for each word and take TFIDF weight instead of average weight.
#Therefore this is TFIDF weighted W2V, But instead of W2V we have used GLOVE